In [1]:
%%time
import math
import torch

from botorch.test_functions import SixHumpCamel
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import (
    ExpectedImprovement,
    ProbabilityOfImprovement,
    qMaxValueEntropy,
)
from botorch.acquisition.max_value_entropy_search import qLowerBoundMaxValueEntropy
import matplotlib.pyplot as plt
from botorch.optim import optimize_acqf
from botorch.acquisition import qNoisyExpectedImprovement, qProbabilityOfImprovement
from time import time
import cProfile, io, pstats
from pstats import SortKey
import scipy

SMOKE_TEST = True

torch.manual_seed(123456)
torch.set_default_dtype(torch.double)

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=1)
scipy.__version__

CPU times: user 2.11 s, sys: 2.24 s, total: 4.35 s
Wall time: 3.88 s


'1.7.3'

In [4]:
%%time

# prep different acqusition functions
candidate_set = torch.rand(
    10000, bounds.size(1), device=bounds.device, dtype=bounds.dtype
)
acqs = {}
# acqs["aa"] = qLowerBoundMaxValueEntropy(model, candidate_set)
# acqs["MES"] = qMaxValueEntropy(model, candidate_set)
acqs["EI"] = qNoisyExpectedImprovement(model, train_X)
# acqs["PI"] = qProbabilityOfImprovement(model, best_f=train_Y.max())


# eval and maximise acq functions
with cProfile.Profile() as pr:
    for acq in acqs.values():
        optimize_acqf(
            acq_function=acq,
            bounds=bounds_norm,
            q=5,
            num_restarts=5,
            raw_samples=5,
            sequential=False,
        )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)

scipy.__version__

         156129 function calls (143631 primitive calls) in 0.171 seconds

   Ordered by: cumulative time
   List reduced from 455 to 46 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.171    0.171 /opt/anaconda3/lib/python3.9/site-packages/botorch/optim/optimize.py:415(optimize_acqf)
        1    0.000    0.000    0.171    0.171 /opt/anaconda3/lib/python3.9/site-packages/botorch/optim/optimize.py:547(_optimize_acqf)
        1    0.000    0.000    0.171    0.171 /opt/anaconda3/lib/python3.9/site-packages/botorch/optim/optimize.py:258(_optimize_acqf_batch)
        1    0.000    0.000    0.165    0.165 /opt/anaconda3/lib/python3.9/site-packages/botorch/optim/optimize.py:294(_optimize_batch_candidates)
        1    0.000    0.000    0.165    0.165 /opt/anaconda3/lib/python3.9/site-packages/botorch/generation/gen.py:43(gen_candidates_scipy)
        1    0.000    0.000    0.164    0.164 /opt/anaconda3/lib/pyth

'1.7.3'